In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import swifter

from pyensembl import EnsemblRelease
from pyensembl.genome import Genome
from pyensembl.shell import collect_all_installed_ensembl_releases
collect_all_installed_ensembl_releases()

/cluster/project/beltrao/gankin/software/miniconda3/envs/vnn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[EnsemblRelease(release=75, species='homo_sapiens'),
 EnsemblRelease(release=111, species='homo_sapiens')]

# Load UKB BIM

In [2]:
data_dir = "/cluster/project/beltrao/gankin/vnn/data"

# Read the .bim file to get SNP information
bim_file = data_dir + "/ukb.bim"
bim_df = pd.read_csv(bim_file, sep='\t', header=None)
bim_df.columns = ['chrom', 'snp', 'cm', 'pos', 'a1', 'a2']
bim_df

/scratch/tmp.20794777.dgankin/ipykernel_4025407/1028760957.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  bim_df = pd.read_csv(bim_file, sep='\t', header=None)


,chrom,snp,cm,pos,a1,a2
0,1,rs28659788,0,723307,C,G
1,1,rs116587930,0,727841,G,A
2,1,rs116720794,0,729632,C,T
3,1,rs3131972,0,752721,A,G
4,1,rs12184325,0,754105,C,T
...,...,...,...,...,...,...
805421,MT,Affx-92047842,0,16337,C,T
805422,MT,Affx-79443531,0,16356,T,C
805423,MT,Affx-79443532,0,16362,T,C
805424,MT,Affx-89025709,0,16390,G,A


# Load Open Targets data

In [3]:
variant_index_parquet = data_dir + "/complete_variant_index.parquet"
variant_index = pd.read_parquet(variant_index_parquet) # takes a few minutes
variant_index

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,gene_id_any_distance,gene_id_any,gene_id_prot_coding_distance,gene_id_prot_coding
0,1,30794,G,T,1,30794,rs1311411458,intron_variant,"{'phred': 0.177, 'raw': -0.402967}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",34625.0,ENSG00000186092,34625.0,ENSG00000186092
1,1,55330,G,A,1,55330,rs185215913,downstream_gene_variant,"{'phred': 0.431, 'raw': -0.275244}","{'gnomad_afr': 0.016711590296495958, 'gnomad_a...",10089.0,ENSG00000186092,10089.0,ENSG00000186092
2,1,57248,C,T,1,57248,rs1249676487,downstream_gene_variant,"{'phred': 7.268, 'raw': 0.304146}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",8171.0,ENSG00000186092,8171.0,ENSG00000186092
3,1,66381,T,A,1,66381,rs538833530,upstream_gene_variant,"{'phred': 1.389, 'raw': -0.101295}","{'gnomad_afr': 0.15321011673151752, 'gnomad_am...",962.0,ENSG00000186092,962.0,ENSG00000186092
4,1,67631,G,C,1,67631,rs533896527,upstream_gene_variant,"{'phred': 0.033, 'raw': -0.632235}","{'gnomad_afr': 0.041812315138478086, 'gnomad_a...",2212.0,ENSG00000186092,2212.0,ENSG00000186092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72878704,X,156006466,G,C,X,155236131,rs1218662062,intron_variant,"{'phred': 1.118, 'raw': -0.133403}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",8770.0,ENSG00000124334,8770.0,ENSG00000124334
72878705,X,156006493,C,T,X,155236158,rs1417497789,intron_variant,"{'phred': 1.776, 'raw': -0.063666}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0011848341...",8797.0,ENSG00000124334,8797.0,ENSG00000124334
72878706,X,156008488,C,T,X,155238153,rs186430584,intron_variant,"{'phred': 3.876, 'raw': 0.076619}","{'gnomad_afr': 0.030184331797235023, 'gnomad_a...",10792.0,ENSG00000124334,10792.0,ENSG00000124334
72878707,X,156009035,CTG,C,X,155238700,rs1381721936,intron_variant,"{'phred': None, 'raw': None}","{'gnomad_afr': 0.0008291873963515755, 'gnomad_...",11339.0,ENSG00000124334,11339.0,ENSG00000124334


# Merge with UKB SNP data

In [4]:
# merge bim with variant index
ot_bim = variant_index.merge(bim_df, left_on='rs_id', right_on='snp', how='inner')
ot_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,gene_id_any_distance,gene_id_any,gene_id_prot_coding_distance,gene_id_prot_coding,chrom,snp,cm,pos,a1,a2
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",4676.0,ENSG00000162571,4676.0,ENSG00000162571,1,rs112695918,0,1104584,T,C
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",12098.0,ENSG00000067606,12098.0,ENSG00000067606,1,rs74372554,0,1969752,C,T
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",18616.0,ENSG00000162585,18616.0,ENSG00000162585,1,rs146607591,0,2125543,A,G
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",6177.0,ENSG00000157870,6177.0,ENSG00000157870,1,rs146100312,0,2524107,C,T
4,1,2828773,C,A,1,2745338,rs116076619,intergenic_variant,"{'phred': 0.058, 'raw': -0.555543}","{'gnomad_afr': 0.0060821666284140465, 'gnomad_...",27080.0,ENSG00000215912,27080.0,ENSG00000215912,1,rs116076619,0,2745338,C,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747754,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",39883.0,ENSG00000102287,39883.0,ENSG00000102287,X,rs145754054,0,151183035,G,A
747755,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",10996.0,ENSG00000183479,10996.0,ENSG00000183479,X,rs5945389,0,152725049,C,T
747756,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",5128.0,ENSG00000184343,5128.0,ENSG00000184343,X,rs199915647,0,153046995,A,C
747757,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",6780.0,ENSG00000269335,6780.0,ENSG00000269335,X,rs5030868,0,153762634,G,A


# Gene ids to gene names

In [5]:
human_data = EnsemblRelease(species='human', release=75)
def get_gene_name(ensembl_id):
    try:
        return human_data.gene_name_of_gene_id(ensembl_id)
    except:
        return None

In [6]:
name_map = pd.DataFrame()
name_map["gene_id"] = ot_bim.gene_id_prot_coding.unique()
name_map["gene_name"]  = name_map["gene_id"].swifter.apply(lambda x: get_gene_name(x))

Pandas Apply: 100%|██████████| 18817/18817 [10:32<00:00, 29.76it/s]


In [7]:
# count None in gene names
print("Gene Id not mappable: ", name_map.gene_name.isnull().sum())
print("Unique gene names: ", name_map.gene_name.nunique())

Gene Id not mappable:  298
Unique gene names:  18510


In [8]:
ot_bim = pd.merge(ot_bim, name_map, left_on='gene_id_prot_coding', right_on='gene_id', how='left')

In [9]:
# remove all None gene names
ot_bim = ot_bim[ot_bim.gene_name.notnull()]
ot_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,gene_id_prot_coding_distance,gene_id_prot_coding,chrom,snp,cm,pos,a1,a2,gene_id,gene_name
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,4676.0,ENSG00000162571,1,rs112695918,0,1104584,T,C,ENSG00000162571,TTLL10
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,12098.0,ENSG00000067606,1,rs74372554,0,1969752,C,T,ENSG00000067606,PRKCZ
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",...,18616.0,ENSG00000162585,1,rs146607591,0,2125543,A,G,ENSG00000162585,C1orf86
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",...,6177.0,ENSG00000157870,1,rs146100312,0,2524107,C,T,ENSG00000157870,FAM213B
4,1,2828773,C,A,1,2745338,rs116076619,intergenic_variant,"{'phred': 0.058, 'raw': -0.555543}","{'gnomad_afr': 0.0060821666284140465, 'gnomad_...",...,27080.0,ENSG00000215912,1,rs116076619,0,2745338,C,A,ENSG00000215912,TTC34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747754,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",...,39883.0,ENSG00000102287,X,rs145754054,0,151183035,G,A,ENSG00000102287,GABRE
747755,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",...,10996.0,ENSG00000183479,X,rs5945389,0,152725049,C,T,ENSG00000183479,TREX2
747756,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",...,5128.0,ENSG00000184343,X,rs199915647,0,153046995,A,C,ENSG00000184343,SRPK3
747757,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",...,6780.0,ENSG00000269335,X,rs5030868,0,153762634,G,A,ENSG00000269335,IKBKG


In [10]:
ot_bim.rs_id.nunique()

731084

In [11]:
ot_bim = ot_bim.drop_duplicates(subset=["rs_id", "gene_name"])
# rename rs_id col to snp
ot_bim = ot_bim.rename({"rs_id":"snp"})
ot_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,gene_id_prot_coding_distance,gene_id_prot_coding,chrom,snp,cm,pos,a1,a2,gene_id,gene_name
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,4676.0,ENSG00000162571,1,rs112695918,0,1104584,T,C,ENSG00000162571,TTLL10
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,12098.0,ENSG00000067606,1,rs74372554,0,1969752,C,T,ENSG00000067606,PRKCZ
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",...,18616.0,ENSG00000162585,1,rs146607591,0,2125543,A,G,ENSG00000162585,C1orf86
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",...,6177.0,ENSG00000157870,1,rs146100312,0,2524107,C,T,ENSG00000157870,FAM213B
4,1,2828773,C,A,1,2745338,rs116076619,intergenic_variant,"{'phred': 0.058, 'raw': -0.555543}","{'gnomad_afr': 0.0060821666284140465, 'gnomad_...",...,27080.0,ENSG00000215912,1,rs116076619,0,2745338,C,A,ENSG00000215912,TTC34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747753,X,151609900,A,G,X,150778372,rs58448897,intron_variant,"{'phred': 3.523, 'raw': 0.055668}","{'gnomad_afr': 0.11482740118046861, 'gnomad_am...",...,46225.0,ENSG00000166049,X,rs58448897,0,150778372,A,G,ENSG00000166049,PASD1
747754,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",...,39883.0,ENSG00000102287,X,rs145754054,0,151183035,G,A,ENSG00000102287,GABRE
747755,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",...,10996.0,ENSG00000183479,X,rs5945389,0,152725049,C,T,ENSG00000183479,TREX2
747756,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",...,5128.0,ENSG00000184343,X,rs199915647,0,153046995,A,C,ENSG00000184343,SRPK3


In [12]:
# get snp_bed_id from bim_df 
bim_df = bim_df.reset_index()
#bim_df.columns = ['snp_bed_id', 'chrom', 'snp', 'cm', 'pos', 'a1', 'a2']
bim_df.columns = ['snp_bed_id', 'chrom', 'snp', 'cm', 'pos', 'a1', 'a2']
bim_df

,snp_bed_id,chrom,snp,cm,pos,a1,a2
0,0,1,rs28659788,0,723307,C,G
1,1,1,rs116587930,0,727841,G,A
2,2,1,rs116720794,0,729632,C,T
3,3,1,rs3131972,0,752721,A,G
4,4,1,rs12184325,0,754105,C,T
...,...,...,...,...,...,...,...
805421,805421,MT,Affx-92047842,0,16337,C,T
805422,805422,MT,Affx-79443531,0,16356,T,C
805423,805423,MT,Affx-79443532,0,16362,T,C
805424,805424,MT,Affx-89025709,0,16390,G,A


In [13]:
ot_bim = ot_bim.merge(bim_df[["snp", "snp_bed_id"]], left_on="snp", right_on="snp")

In [14]:
ot_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,gene_id_prot_coding,chrom,snp,cm,pos,a1,a2,gene_id,gene_name,snp_bed_id
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,ENSG00000162571,1,rs112695918,0,1104584,T,C,ENSG00000162571,TTLL10,220
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,ENSG00000067606,1,rs74372554,0,1969752,C,T,ENSG00000067606,PRKCZ,599
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",...,ENSG00000162585,1,rs146607591,0,2125543,A,G,ENSG00000162585,C1orf86,648
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",...,ENSG00000157870,1,rs146100312,0,2524107,C,T,ENSG00000157870,FAM213B,838
4,1,2828773,C,A,1,2745338,rs116076619,intergenic_variant,"{'phred': 0.058, 'raw': -0.555543}","{'gnomad_afr': 0.0060821666284140465, 'gnomad_...",...,ENSG00000215912,1,rs116076619,0,2745338,C,A,ENSG00000215912,TTC34,889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731079,X,151609900,A,G,X,150778372,rs58448897,intron_variant,"{'phred': 3.523, 'raw': 0.055668}","{'gnomad_afr': 0.11482740118046861, 'gnomad_am...",...,ENSG00000166049,X,rs58448897,0,150778372,A,G,ENSG00000166049,PASD1,802181
731080,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",...,ENSG00000102287,X,rs145754054,0,151183035,G,A,ENSG00000102287,GABRE,802336
731081,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",...,ENSG00000183479,X,rs5945389,0,152725049,C,T,ENSG00000183479,TREX2,802547
731082,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",...,ENSG00000184343,X,rs199915647,0,153046995,A,C,ENSG00000184343,SRPK3,802648


# Load GO term to gene mapping

In [2]:
import networkx as nx
import pickle 

In [34]:
#go_path = "../../data/GO_biological_process.pickle"
go_path = "../../ontology_operations/data/GO_cellular_component.pickle"
G = pickle.load(open(go_path, 'rb'))

In [3]:
# Load go terms and genes
#goterms_df = pd.read_csv("../../ontology_operations/data/GO_terms_biological_processes_merged_EBI_uniprot.csv", index_col=0)
goterms_df = pd.read_csv("../../ontology_operations/data/GO_terms_cellular_components_merged_EBI_uniprot.csv", index_col=0)

In [4]:
goterms_df["gene_name"] = goterms_df["merged"].apply(lambda x: x.split(";"))
goterms_df = goterms_df.explode("gene_name")
goterms_df

,merged,len_merged,name,gene_name
GO:0000137,GLP;GOLGA8S;GOLGA6A;HUGL;GOT1B;GOLGA6B;GOLGA;N...,49,Golgi cis cisterna,GLP
GO:0000137,GLP;GOLGA8S;GOLGA6A;HUGL;GOT1B;GOLGA6B;GOLGA;N...,49,Golgi cis cisterna,GOLGA8S
GO:0000137,GLP;GOLGA8S;GOLGA6A;HUGL;GOT1B;GOLGA6B;GOLGA;N...,49,Golgi cis cisterna,GOLGA6A
GO:0000137,GLP;GOLGA8S;GOLGA6A;HUGL;GOT1B;GOLGA6B;GOLGA;N...,49,Golgi cis cisterna,HUGL
GO:0000137,GLP;GOLGA8S;GOLGA6A;HUGL;GOT1B;GOLGA6B;GOLGA;N...,49,Golgi cis cisterna,GOT1B
...,...,...,...,...
GO:1903754,KIAA0627;CLASP2,2,cortical microtubule plus-end,CLASP2
GO:1990836,HSPA10;HSPA8;HSC70;HSP73,4,lysosomal matrix,HSPA10
GO:1990836,HSPA10;HSPA8;HSC70;HSP73,4,lysosomal matrix,HSPA8
GO:1990836,HSPA10;HSPA8;HSC70;HSP73,4,lysosomal matrix,HSC70


# Filter genes

In [18]:
gene_df = pd.DataFrame()
gene_df["gene_name"] = goterms_df.gene_name.unique()

In [19]:
go_bim = ot_bim.merge(gene_df, on="gene_name")

In [20]:
go_bim.gene_name.nunique() # some genes not mapped to go terms ..

16042

In [40]:
go_bim.gene_name.nunique() # some genes not mapped to go terms ..

16042

In [21]:
go_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,gene_id_prot_coding,chrom,snp,cm,pos,a1,a2,gene_id,gene_name,snp_bed_id
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,ENSG00000162571,1,rs112695918,0,1104584,T,C,ENSG00000162571,TTLL10,220
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,ENSG00000067606,1,rs74372554,0,1969752,C,T,ENSG00000067606,PRKCZ,599
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",...,ENSG00000162585,1,rs146607591,0,2125543,A,G,ENSG00000162585,C1orf86,648
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",...,ENSG00000157870,1,rs146100312,0,2524107,C,T,ENSG00000157870,FAM213B,838
4,1,3041036,T,C,1,2957600,rs12409277,intergenic_variant,"{'phred': 1.823, 'raw': -0.059547}","{'gnomad_afr': 0.13893357848770396, 'gnomad_am...",...,ENSG00000169717,1,rs12409277,0,2957600,T,C,ENSG00000169717,ACTRT2,958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644889,X,151609900,A,G,X,150778372,rs58448897,intron_variant,"{'phred': 3.523, 'raw': 0.055668}","{'gnomad_afr': 0.11482740118046861, 'gnomad_am...",...,ENSG00000166049,X,rs58448897,0,150778372,A,G,ENSG00000166049,PASD1,802181
644890,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",...,ENSG00000102287,X,rs145754054,0,151183035,G,A,ENSG00000102287,GABRE,802336
644891,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",...,ENSG00000183479,X,rs5945389,0,152725049,C,T,ENSG00000183479,TREX2,802547
644892,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",...,ENSG00000184343,X,rs199915647,0,153046995,A,C,ENSG00000184343,SRPK3,802648


In [22]:
gene_df = pd.DataFrame()
gene_df["gene_name"] = go_bim.gene_name.unique()
gene_df.reset_index(inplace=True)
gene_df.columns = ["gene_bed_id", "gene_name"]
gene_df

,gene_bed_id,gene_name
0,0,TTLL10
1,1,PRKCZ
2,2,C1orf86
3,3,FAM213B
4,4,ACTRT2
...,...,...
16037,16037,RPL18
16038,16038,HOXA3
16039,16039,TAF6
16040,16040,TPRN


In [23]:
go_bim = go_bim.merge(gene_df, on="gene_name")
go_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,chrom,snp,cm,pos,a1,a2,gene_id,gene_name,snp_bed_id,gene_bed_id
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,1,rs112695918,0,1104584,T,C,ENSG00000162571,TTLL10,220,0
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,1,rs74372554,0,1969752,C,T,ENSG00000067606,PRKCZ,599,1
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",...,1,rs146607591,0,2125543,A,G,ENSG00000162585,C1orf86,648,2
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",...,1,rs146100312,0,2524107,C,T,ENSG00000157870,FAM213B,838,3
4,1,3041036,T,C,1,2957600,rs12409277,intergenic_variant,"{'phred': 1.823, 'raw': -0.059547}","{'gnomad_afr': 0.13893357848770396, 'gnomad_am...",...,1,rs12409277,0,2957600,T,C,ENSG00000169717,ACTRT2,958,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644889,X,151609900,A,G,X,150778372,rs58448897,intron_variant,"{'phred': 3.523, 'raw': 0.055668}","{'gnomad_afr': 0.11482740118046861, 'gnomad_am...",...,X,rs58448897,0,150778372,A,G,ENSG00000166049,PASD1,802181,9628
644890,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",...,X,rs145754054,0,151183035,G,A,ENSG00000102287,GABRE,802336,7660
644891,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",...,X,rs5945389,0,152725049,C,T,ENSG00000183479,TREX2,802547,2413
644892,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",...,X,rs199915647,0,153046995,A,C,ENSG00000184343,SRPK3,802648,12949


In [24]:
all_snp_ids = go_bim.snp_bed_id.unique() # list of all snp ids, enumerate and map to ids
snp_id_dict = dict(zip(all_snp_ids, range(len(all_snp_ids))))
# snp position in gene_level bed file
go_bim['snp_gbed_id'] = go_bim['snp_bed_id'].map(snp_id_dict)

In [25]:
go_bim[["snp", "gene_name", "snp_bed_id", "snp_gbed_id", "gene_bed_id"]]

,snp,gene_name,snp_bed_id,snp_gbed_id,gene_bed_id
0,rs112695918,TTLL10,220,0,0
1,rs74372554,PRKCZ,599,1,1
2,rs146607591,C1orf86,648,2,2
3,rs146100312,FAM213B,838,3,3
4,rs12409277,ACTRT2,958,4,4
...,...,...,...,...,...
644889,rs58448897,PASD1,802181,644889,9628
644890,rs145754054,GABRE,802336,644890,7660
644891,rs5945389,TREX2,802547,644891,2413
644892,rs199915647,SRPK3,802648,644892,12949


In [26]:
go_bim[["snp", "gene_name", "snp_bed_id", "snp_gbed_id", "gene_bed_id"]].to_csv("../data/go_cellular_components_bim.csv", index=False, sep="\t")

In [ ]:
#go_bim.to_csv("../data/go_biological_processes_bim_full.csv", index=False, sep="\t")

# Prune network

In [5]:
import networkx as nx
import pickle

#go_path = "../../ontology_operations/data/GO_biological_process.pickle"
go_path = "../../ontology_operations/data/GO_cellular_component_trimmed.pickle"
G = pickle.load(open(go_path, 'rb'))

In [6]:
go_bim = pd.read_csv("../data/go_biological_processes_bim_full.csv", sep="\t")

In [7]:
merged_go = go_bim.merge(goterms_df.reset_index(), on="gene_name") # .... lot's of multiple connections

In [29]:
merged_go

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,a2,gene_id,gene_name,snp_bed_id,gene_bed_id,snp_gbed_id,index,merged,len_merged,name
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,C,ENSG00000162571,TTLL10,220,0,0,GO:0005829,M130;NFATC1;GGCT;PKDL;TORC3;NIP2;CPGL;KNSL5;DN...,13464,cytosol
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,T,ENSG00000067606,PRKCZ,599,1,1,GO:0001725,C20orf28;CTTNBP2NL;LPP;KIAA1296;KIAA0991;DRR1;...,211,stress fiber
2,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,T,ENSG00000067606,PRKCZ,599,1,1,GO:0005634,DHX38;B4E246;NFATC1;CRAMP1L;ZNF526;CCNP;TORC3;...,17387,nucleus
3,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,T,ENSG00000067606,PRKCZ,599,1,1,GO:0005737,LSLCL;NFATC1;CCNP;TORC3;NIP2;B2R7M0;ATG16L2;GO...,16823,cytoplasm
4,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,T,ENSG00000067606,PRKCZ,599,1,1,GO:0005768,KIAA0319;A0A384ME16;PKN;ARL8B;B3KQD2;B4E2S7;TM...,1498,endosome
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2946977,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",...,A,ENSG00000269335,IKBKG,802968,13478,644893,GO:0005634,DHX38;B4E246;NFATC1;CRAMP1L;ZNF526;CCNP;TORC3;...,17387,nucleus
2946978,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",...,A,ENSG00000269335,IKBKG,802968,13478,644893,GO:0005654,DHX38;NFATC1;TTF1;TORC3;CPGL;C20orf77;PRO1146;...,10042,nucleoplasm
2946979,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",...,A,ENSG00000269335,IKBKG,802968,13478,644893,GO:0005737,LSLCL;NFATC1;CCNP;TORC3;NIP2;B2R7M0;ATG16L2;GO...,16823,cytoplasm
2946980,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",...,A,ENSG00000269335,IKBKG,802968,13478,644893,GO:0005829,M130;NFATC1;GGCT;PKDL;TORC3;NIP2;CPGL;KNSL5;DN...,13464,cytosol


In [8]:
merged_go["index"].nunique()

859

In [11]:
merged_go["index"].nunique()

11364

In [14]:
go_terms = merged_go["index"].unique()

In [9]:
go_terms = {i.replace(":", "_") for i in go_terms}
go_terms

NameError: name 'go_terms' is not defined

In [14]:
# only keep terms that are also in the graph
graph_terms = {i.replace("_",":") for i in G.nodes}
merged_go = merged_go[merged_go["index"].isin(graph_terms)]

In [12]:
graph_terms

{'GO_0005619',
 'GO_0044222',
 'GO_0009570',
 'GO_0070111',
 'GO_0034515',
 'GO_0098685',
 'GO_0160211',
 'GO_0110165',
 'GO_0000933',
 'GO_0042648',
 'GO_0044173',
 'GO_0000328',
 'GO_0098569',
 'GO_0035841',
 'GO_0070113',
 'GO_0061474',
 'GO_0034774',
 'GO_0030426',
 'GO_0043025',
 'GO_0000801',
 'GO_0001651',
 'GO_1905754',
 'GO_0002081',
 'GO_0032982',
 'GO_0031256',
 'GO_0042584',
 'GO_0001740',
 'GO_0033653',
 'GO_0045495',
 'GO_0098992',
 'GO_0097564',
 'GO_0034469',
 'GO_0061174',
 'GO_0061689',
 'GO_0030486',
 'GO_0031912',
 'GO_0099192',
 'GO_0036186',
 'GO_0036012',
 'GO_0098954',
 'GO_0030061',
 'GO_0085042',
 'GO_0061823',
 'GO_0005916',
 'GO_0070114',
 'GO_0005714',
 'GO_0098520',
 'GO_0044195',
 'GO_0044296',
 'GO_0031972',
 'GO_0032160',
 'GO_0044841',
 'GO_0097618',
 'GO_0005776',
 'GO_0060473',
 'GO_0039717',
 'GO_0042717',
 'GO_0009569',
 'GO_0031977',
 'GO_0000242',
 'GO_0039721',
 'GO_0016606',
 'GO_0044201',
 'GO_0032590',
 'GO_0070057',
 'GO_0072324',
 'GO_00316

In [16]:
print("Nodes before pruning:", len(G.nodes)) # 24k

Nodes before pruning: 1571


In [34]:
# prune network
def prune_tree(G, nodeset):
    nodes_to_check = list(G.nodes)
    while nodes_to_check:
        node = nodes_to_check.pop()
        if G.in_degree(node) == 0 and G.out_degree(node) > 0 and not node in nodeset: # if leaf is not in nodeset (leaf meaning )
            parents = list(G.predecessors(node))
            G.remove_node(node)
            nodes_to_check.extend(parents)

prune_tree(G, go_terms)

print("Number of nodes left", len(G.nodes))

Number of nodes left 1103


In [26]:
list(nx.topological_sort(G))[-5:]

['GO_0031974', 'GO_0033643', 'GO_0043226', 'GO_0018995', 'GO_0110165']

{'GO_0018995', 'GO_0110165'}

In [17]:
list(nx.topological_sort(G))[-5:]

['GO_0033643', 'GO_0043226', 'GO_0018995', 'GO_0110165', 'GO_root']

In [20]:
len([n for n in G.nodes if G.out_degree(n) == 0])

1

# Build full ontology

In [15]:
# Convert the pruned graph to a dataframe for output
pruned_edges = [(v,u) for u, v in G.edges]
ontology_df = pd.DataFrame(pruned_edges, columns=['parent', 'child'])
ontology_df["type"] = "default"
ontology_df

# append gene to go term df
tmp_df = merged_go[["index","gene_name"]].drop_duplicates()
tmp_df['index'] = tmp_df['index'].str.replace(":","_")
tmp_df.columns = ['parent', 'child']
tmp_df['type'] = 'gene'

# append snp to gene df
snp_df = go_bim[["gene_name","snp"]]
snp_df.columns = ['parent', 'child']
snp_df['type'] = 'snp'

# concatenate
ontology_df = pd.concat([ontology_df, tmp_df, snp_df])
ontology_df

/scratch/tmp.20832073.dgankin/ipykernel_35837/3662680718.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snp_df['type'] = 'snp'


,parent,child,type
0,GO_0031985,GO_0000137,default
1,GO_0031985,GO_0000138,default
2,GO_0098588,GO_0000139,default
3,GO_0000399,GO_0000144,default
4,GO_0032161,GO_0000144,default
...,...,...,...
644889,PASD1,rs58448897,snp
644890,GABRE,rs145754054,snp
644891,TREX2,rs5945389,snp
644892,SRPK3,rs199915647,snp


In [21]:
# Create a directed graph from the ontology data
OG = nx.DiGraph()

# Add edges to the graph
for index, row in ontology_df.iterrows():
    OG.add_edge(row['parent'], row['child'], type=row['type']) # only add gene if in known genes

print("Nodes in Graph:", len(OG.nodes))
print("Edges in Graph:", len(OG.edges))

#print(G.nodes)

# Step 2: Iteratively remove nodes that do not lead to a known gene
def prune_tree_(G):
    nodes_to_check = list(G.nodes)
    while nodes_to_check:
        node = nodes_to_check.pop()
        if G.out_degree(node) == 0 and G.in_degree(node) > 0 and "GO" in node:  # Check if it is a hanging node
            parents = list(G.predecessors(node))
            G.remove_node(node)
            nodes_to_check.extend(parents)

prune_tree_(OG)

print("Number of nodes left", len(OG.nodes))

# Convert the pruned graph back to a dataframe for output
pruned_edges = [(u, v, OG[u][v]['type']) for u, v in OG.edges]
pruned_df = pd.DataFrame(pruned_edges, columns=['parent', 'child', 'type'])

# Save the pruned ontology
#pruned_ontology_file = '../data/go_biological_processes.txt'
pruned_ontology_file = '../data/go_cellular_components.txt'
pruned_df.to_csv(pruned_ontology_file, sep='\t', index=False, header=False)

pruned_df.head()

Nodes in Graph: 662507
Edges in Graph: 713784
Number of nodes left 661646


,parent,child,type
0,GO_0031985,GO_0000137,default
1,GO_0031985,GO_0000138,default
2,GO_0031985,GO_0005797,default
3,GO_0031985,STX16,gene
4,GO_0031985,SORL1,gene


In [38]:
pruned_df

,parent,child,type
0,GO_0031985,GO_0000137,default
1,GO_0031985,GO_0000138,default
2,GO_0031985,GO_0005797,default
3,GO_0031985,STX16,gene
4,GO_0031985,SORL1,gene
...,...,...,...
716316,RPL18,rs12608448,snp
716317,HOXA3,rs118016485,snp
716318,TAF6,rs143557149,snp
716319,TPRN,rs144769563,snp


In [75]:
# OG
list(nx.topological_sort(OG))

['GO_root',
 'GO_0005874',
 'GO_0042622',
 'GO_0005884',
 'GO_0031234',
 'GO_0042383',
 'GO_0062023',
 'GO_0005682',
 'GO_0071007',
 'GO_0071013',
 'GO_1990877',
 'GO_0071001',
 'GO_0005879',
 'GO_0036126',
 'GO_0042470',
 'GO_0002096',
 'GO_0005881',
 'GO_0034709',
 'GO_0005686',
 'GO_0071005',
 'GO_0071011',
 'GO_0001533',
 'GO_0005882',
 'GO_0008091',
 'GO_0072534',
 'GO_0000221',
 'GO_0031941',
 'GO_0005883',
 'GO_0030686',
 'GO_0032040',
 'GO_0005581',
 'GO_0005600',
 'GO_0015935',
 'GO_0044391',
 'GO_0097431',
 'GO_0005838',
 'GO_0005905',
 'GO_0030117',
 'GO_0005762',
 'GO_0005915',
 'GO_0008282',
 'GO_0097342',
 'GO_1990498',
 'GO_0097729',
 'GO_0098850',
 'GO_1990909',
 'GO_0045095',
 'GO_0097539',
 'GO_0030688',
 'GO_0099147',
 'GO_0008180',
 'GO_0098890',
 'GO_0005587',
 'GO_0000145',
 'GO_0034045',
 'GO_0005685',
 'GO_0005687',
 'GO_0071004',
 'GO_0001527',
 'GO_0016591',
 'GO_0005862',
 'GO_0097524',
 'GO_0033018',
 'GO_0030125',
 'GO_0005763',
 'GO_0036157',
 'GO_0034361'

In [ ]:
OG.out_edges("")

In [60]:
OG.in_edges('GO_0033180')

InEdgeDataView([])

In [62]:
len([n for n in OG.nodes if OG.in_degree(n) == 0])

141

In [63]:
len([n for n in G.nodes if G.out_degree(n) == 0])

268

In [1]:
import pandas as pd 

In [4]:
pd.read_csv("../data/go_cellular_components.txt", sep="\t")#.to_csv("../data/go_cellular_components.txt", sep='\t', index=False, header=False)

,GO_0000137,GO_0031985,default
0,GO_0000137,GOLGA8J,gene
1,GO_0000137,GOLGA6A,gene
2,GO_0000137,XYLT1,gene
3,GO_0000137,ATL1,gene
4,GO_0000137,GOLGA8K,gene
...,...,...,...
716629,RPL18,rs12608448,snp
716630,HOXA3,rs118016485,snp
716631,TAF6,rs143557149,snp
716632,TPRN,rs144769563,snp
